In [1]:
import random
import numpy as np
from  itertools import combinations
from scipy import stats
import statsmodels
import statsmodels.stats

import statsmodels.stats.multitest as sm

In [2]:
import pandas as pd

In [3]:
import pprint
pp = pprint.PrettyPrinter(indent = 5)

In [4]:
def resample(l):
    final = []
    for i in range(len(l)):
        final.append(random.choice(l))
    return final

def repeat_resample(sample_a, sample_b, num_iter = 1000):
    difference_in_means = []#keep track of the difference in heights for each experiment
    for i in range(num_iter):
        resample_a = resample(sample_a)
        resample_b = resample(sample_b)
        difference = np.mean(resample_a) - np.mean(resample_b)
        difference_in_means.append(difference)
    return difference_in_means

def comb_samples_to_pop(*args):
    all_ = []
    for i in args:
        all_.extend(i)
    return all_


In [5]:
def anova_omnibus( *args, num_iter = 1000, verbose = False):
    def get_var_bet(*args):
        all_means = []
        for  i in args:
            all_means.append(np.mean(i))
        return np.var(all_means)   
    pop = comb_samples_to_pop(*args)
    var_between_groups = get_var_bet(*args)
    if verbose:
        print(var_between_groups)
    resampled_vars = []
    for i in range(num_iter):
        random.shuffle(pop)
        means_of_groups = []
        for i in args:
            s = random.choices(pop, k = len(i))
            means_of_groups.append(np.mean(s))
        resampled_vars.append(np.var(means_of_groups))
    diff = [var_between_groups - x for x in resampled_vars]
    p_value =  1 - len([x for x in diff if x > 0])/len(diff)
    return p_value

def test_ominbus():
    
    s1 = [random.gauss(0,1) for x in range(100)]
    s2 = [random.gauss(0,1) for x in range(100)]
    s3 = [random.gauss(.3,1) for x in range(100)]
    p_mine = anova_omnibus(s1, s2, s3, verbose = False, num_iter = 1000) 
    F, p = stats.f_oneway(s1, s2, s3 )
    return p_mine, p
   
diffs =[]
for i in range(100):
    p_mine, p_theory = test_ominbus()
    reject_mine = p_mine < .05
    reject_theory = p_theory < .05
    if reject_mine != reject_theory:
        print(p_mine, p_theory)

    diffs.append(p_mine - p_theory)
np.mean(diffs)

0.052000000000000046 0.04887071763614726
0.04400000000000004 0.05908152105447243


0.0009521250440731675

In [6]:
def get_data():
    perm_mold = [45.5, 45.3, 45.4, 44.4, 44.6, 43.9, 44.6, 44.0]
    die_casting = [44.2, 43.9, 44.7, 44.2, 44, 43.8, 44.6, 43.1]
    plast_mold = [46, 45.9, 44.8, 46.2, 45.1, 45.5]
    df = pd.DataFrame([perm_mold, die_casting, plast_mold ],  index = ['perm_mold', 'die_casting', 'plast_mold'])
    return df
def test_from_book():
    data = get_data()
    s1 = data.loc['perm_mold']
    s2 = data.loc['die_casting']
    s3 = data.loc['plast_mold'].dropna()
    p_mine = anova_omnibus(s1, s2, s3)
    F, p_theory = stats.f_oneway(s1, s2, s3 )
    print(p_mine, p_theory)

test_from_book()   


0.0020000000000000018 0.0003336160734123371


In [7]:
def compare_samples(the_dict, correction_method ='fdr_bh' ):
    assert isinstance(the_dict, dict)
    final = []
    final_d = {}
    for i in combinations(the_dict.keys(), 2):
        s1 = the_dict[i[0]]
        s2 = the_dict[i[1]]
        p_value = stats.ttest_ind(s1, s2).pvalue
        final.append([i, p_value])
    p_values = [x[1] for x in final]
    corrected = sm.multipletests(p_values, method = correction_method)
    rejects = corrected[0]
    p_adj = corrected[1]
    for counter, i in enumerate(final):
        final_d[i[0]] = {'p_value': i[1], 'reject':rejects[counter], 'adj_p': p_adj[counter]}
    return final_d

def test_compare_samples(num_iter = 100, correction_method = 'fdr_bh'):
    s1_s2 = []
    s1_s2_nc = []
    s1_s3 = []
    for  i in range(num_iter):
        s1 = [random.gauss(0,1) for x in range(100)]
        s2 = [random.gauss(0,1) for x in range(100)]
        s3 = [random.gauss(.3,1) for x in range(100)]
        #skipping omnibus
        
        result = compare_samples(
            {'s1': s1, 
                's2': s2,
                's3': s3
            }, correction_method = correction_method)
        s1_s2.append(result[('s1', 's2')]['reject'])
        s1_s3.append(result[('s1', 's3')]['reject'])
        s1_s2_nc.append(result[('s1', 's2')]['p_value'] < .05)
    print('s1 s2 sig is {s}'.format(
        s = len([x for x in s1_s2 if x])))
    print('s1 s2 sig non correct is {s}'.format(
        s = len([x for x in s1_s2_nc if x])))
    print('s1 s3 sig is {s}'.format(
        s = len([x for x in s1_s3 if x])))

test_compare_samples(100)

s1 s2 sig is 5
s1 s2 sig non correct is 6
s1 s3 sig is 49


In [8]:
def test_correction_factor():
    p_values = []
    for i in range(100):
        s1 = [random.gauss(0,1) for x in range(100)]
        s2 = [random.gauss(0,1) for x in range(100)]
        p_values.append(stats.ttest_ind(s1, s2).pvalue)
    print('non corrected sig is {l}'.format(
        l = len([x for x in p_values if x < .05])))
    corrected_fdr = sm.multipletests(p_values, method = 'fdr_bh')[0]
    corrected_bonferroni = sm.multipletests(p_values, method = 'bonferroni')[0]
    print('corrected with fdr sig is {l}'.format(
        l = len([x for x in corrected_fdr if x])))
    print('corrected with bonferroni sig is {l}'.format(
        l = len([x for x in corrected_bonferroni if x])))
test_correction_factor()
    

non corrected sig is 10
corrected with fdr sig is 0
corrected with bonferroni sig is 0


In [9]:
def fdr(p_values, alpha = .05):
    final = []
    s = sorted(p_values)
    for counter, i in enumerate(s):
        a_p = round(alpha * (counter + 1)/len(p_values),2)
        tp = round(i * len(p_values)/(counter + 1),4)
        if i < a_p:
            final.append(True)
        else:
            final.append(False)
        print(i, a_p, tp)
    return final
method = 'bonferroni'
method = 'fdr_bh'
p_values = [.08, .001, .003, .05, .06,  .088]
p_adj = sm.multipletests(p_values, method = method)
print(p_adj)
print(fdr(p_values))

(array([False,  True,  True, False, False, False]), array([0.088, 0.006, 0.009, 0.088, 0.088, 0.088]), 0.008512444610847103, 0.008333333333333333)
0.001 0.01 0.006
0.003 0.02 0.009
0.05 0.03 0.1
0.06 0.03 0.09
0.08 0.04 0.096
0.088 0.05 0.088
[True, True, False, False, False, False]
